In [ ]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import RandomSampler
import warnings
warnings.filterwarnings("ignore")
import multiprocessing
max_n_jobs = multiprocessing.cpu_count()
print(f"Maximum n_jobs you can use: {max_n_jobs}")

In [ ]:
feature_version = 2
default_cv = 1
# 1 for pc feature, 
# 2 for label correlation feature
# 3 for best features based on combination rank

Train the best model

In [ ]:
train_df = pd.read_parquet(f"data/cleaned/cleaned_train_{feature_version}.parquet")
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"])
train_df = train_df[train_df["timestamp"].dt.month.isin([1, 2, 12, 11, 10, 9])]
X_train = train_df.drop(columns=["timestamp", "label"])
Y_train = train_df["label"]

In [ ]:
default_n_trees = 1000
default_random_state = 101

In [ ]:
def get_best_params_from_file(filename):
    study = optuna.load_study(
        study_name = filename,
        storage = f"sqlite:///{filename}.db"
    )
    return study.best_params

In [ ]:
params = {
    "n_estimators": default_n_trees,
    "verbosity": 0,
    "enable_categorical": True,
    "random_state": default_random_state
}
best_params_xgboost = get_best_params_from_file(f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study")
for p in best_params_xgboost:
    params[p] = best_params_xgboost[p]

xgbr = XGBRegressor(**params)
xgbr.fit(X_train, Y_train)

In [ ]:
# params = {
#     "n_estimators": default_n_trees,
#     "verbosity": -1,
#     "random_state": default_random_state
# }
# best_params_lightgbm = get_best_params_from_file(f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study")
# for p in best_params_lightgbm:
#     params[p] = best_params_lightgbm[p]

# lgbr = LGBMRegressor(**params)
# lgbr.fit(X_train, Y_train)

Making prediction & submission

In [ ]:
X_test = pd.read_parquet(f"data/cleaned/cleaned_test_{feature_version}.parquet")
X_test = X_test.drop(columns=["label"])
Y_pred = xgbr.predict(X_test)

In [ ]:
submission = pd.DataFrame({
    "id": X_test.index + 1,
    "prediction": Y_pred
})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)